# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
01.06.18	Metacognition of visuomotor decisions in conversion disorder	Neuropsychologia		"Indrit Begue, Rebekah Blakemore, Julian Klug, Yann Cojan, Silvio Galli, Alexandre Berneye, Selma Aybek, Patrik Vuilleumier. Metacognition of visuomotor decisions in conversion disorder. Neuropsychologia, Volume 114, June 2018."	paper-metacognition-conversion	https://www.ncbi.nlm.nih.gov/pubmed/29698734
01.02.19	Three-dimensional Surface Reconstruction of the Human Cochlear Nucleus: Implications for the ABI. 	Otolaryngology - Head and Neck Surgery		"Vivek V. Kanumuri, Osama Tarabichi, Julian Klug, Nicolas Vachicouras, Maria J. Duarte, Elliott D. Kozin, Stephanie P. Lacour, M. Christian Brown, Daniel J. Lee. Three-dimensional Surface Reconstruction of the Human Cochlear Nucleus: Implications for the ABI. Otolaryngology - Head and Neck Surgery, February 2019."	paper-3D-CN-ABI	https://www.thieme-connect.com/products/ejournals/abstract/10.1055/s-0

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0, encoding = "ISO-8859-1")
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,01.06.18,Metacognition of visuomotor decisions in conve...,Neuropsychologia,NaN,"Indrit Begue, Rebekah Blakemore, Julian Klug, ...",paper-metacognition-conversion,https://www.ncbi.nlm.nih.gov/pubmed/29698734
1,01.02.19,Three-dimensional Surface Reconstruction of th...,Otolaryngology - Head and Neck Surgery,NaN,"Vivek V. Kanumuri, Osama Tarabichi, Julian Klu...",paper-3D-CN-ABI,https://www.thieme-connect.com/products/ejourn...
2,05.06.20,Integrating regional perfusion CT information ...,Journal of Cerebral Blood Flow & Metabolism,NaN,"Klug, J., Dirren, E., Preti, M. G., Machi, P.,...",paper-regional-perfusion-CT,https://journals.sagepub.com/doi/10.1177/02716...
3,27.03.21,Bayesian Skip Net: Building on Prior Informati...,"Brainlesion: Glioma, Multiple Sclerosis, Strok...",NaN,"Klug J., Leclerc G., Dirren E., Preti M.G., Va...",paper-bayesian-skip-net,https://doi.org/10.1007/978-3-030-72084-1_16


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'wb') as f:
        f.write(md.encode('utf-8'))

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

01.02.19-paper-3D-CN-ABI.md
01.06.18-paper-metacognition-conversion.md
05.06.20-paper-regional-perfusion-CT.md
2018-06-01-paper-metacognition-conversion.md
2019-02-01-paper-3D-CN-ABI.md
2020-06-05-paper-regional-perfusion-CT.md
27.03.21-paper-bayesian-skip-net.md


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
